In [326]:
import random
import itertools
import operator
import numpy as np 
import scipy as sp
import scipy.linalg as spla
np.set_printoptions(precision=4, linewidth=np.nan)

In [361]:
class LPSolution(object):
    
    def __init__(self, num_vars=0, var_vals=list(), obj=0):
        
        self.num_vars = num_vars
        self.obj = obj
        self.var_vals = var_vals
        
    def __str__(self):
        
        sol = ""
        sol += "\tSolution to the LP is as follows:\n\n"
        sol += "\t\tc'x*\t=\t" + str(self.obj) + "\n\n"
        for i in range(self.num_vars):
            if i in self.var_vals:
                sol += "\t\tx_" + str(i + 1) + "\t=\t" + str(self.var_vals[i]) + "\n"
            else:
                sol += "\t\tx_" + str(i + 1) + "\t=\t" + str(0.0) + "\n"
        return sol

In [424]:
class Simplex(object):
    # num_eq_constraints : no. of equality constraints
    def __init__(self, num_eq_constraints, num_vars, objective, constraints=None, max_iter=10):

        self.num_eq_constraints = num_eq_constraints
        self.num_vars = num_vars
        self.c = objective
        if constraints is not None:
            self.b = self.constraints[:self.num_eq_constraints, -1]
            self.constraints  = self.constraints[:, :-1]
            self.A = self.constraints[:self.num_eq_constraints, :]
            self.get_nondg_basic_columns()
        else:
            self.A = None
            self.b = None
            self.basic_columns = None
        self.solution = None
        self.tableau = None
        self.max_iter = max_iter
        
    
    def set_constraints(self, const):
        self.constraints = const
    
    def get_num_constraints(self):
        return self.num_constraints
    
    def get_num_vars(self):
        return self.num_vars
    
    def get_constraints(self):
        return self.constraints
    
    
    def fetch_constraints(self, filename):
        self.constraints = np.loadtxt(filename)
        self.b = self.constraints[:self.num_eq_constraints, -1]
        self.constraints  = self.constraints[:, :-1]
        self.A = self.constraints[:self.num_eq_constraints, :]
        self.get_nondg_basic_columns()
    
    def get_sol(self):
        return self.sol
    
    def get_nondg_basic_columns(self):
        basic_columns = None
        for i in itertools.combinations(range(self.A.shape[1]), self.A.shape[0]):
            B_poss = self.A[:, list(i)]                       # Possible B
#             print(B_poss)
            pivots = np.linalg.matrix_rank(B_poss)
            if pivots == B_poss.shape[1]:
                if basic_columns is None:
                    basic_columns = list()
                x_b = np.linalg.solve(B_poss, self.b)
                if (x_b > 0).all():
                    basic_columns.append(list(i))
                    if self.A.shape[0] > 50 and len(basic_columns) == 1:
                        break
                else:
                    continue
                

        if basic_columns is None:
            raise RuntimeError("No initial non-degenrate BFS detected! Terminating algorithm...")
#         basic_columns = random.choice(basic_columns)
        self.basic_columns = basic_columns
    
    def get_first_basic_columns(self):
        basic_columns = random.choice(self.basic_columns)
        return basic_columns
    
    def get_first_B(self, basic_cols):
        return self.A[:, basic_cols]
    
    def form_tableau(self):
        basic_columns = self.get_first_basic_columns()
        b = self.b
        B = self.get_first_B(basic_columns)
        c_B = self.c[basic_columns]
        zeroth_element = -1 * np.dot(c_B, np.linalg.solve(B, b))
        zeroth_col = np.linalg.solve(B, b)
#         print(B)
        zeroth_row = self.c - np.dot(c_B, np.dot(np.linalg.inv(B), self.A))
        rest = np.dot(np.linalg.inv(B), self.A)
        tableau = np.block([
            [zeroth_element, zeroth_row],
            [zeroth_col.reshape(1, zeroth_col.shape[0]).T, rest]
        ])
        return tableau, basic_columns
    
#     def display_tableau(self, basic_cols, tableau):
#         print("\t\t")
#         for i in tableau:
#             for 
    
    def run_simplex(self):
        while True:
            tableau, basic_columns = self.form_tableau()
            self.tableau = tableau.copy()
            print(tableau)
            iters = 0
            flag = 0                                        # indicates if we have the right initial BFS or not
            while (tableau[0, 1:] < 0).any():
                j = np.where(tableau[0, 1:] < 0)[0][0]     # incoming basis direction
                theta = [i for i in range(1, tableau.shape[0]) if tableau[i, j + 1] > 0][0]
                for i in range(1, tableau.shape[0]): 
                    if tableau[i, j + 1] > 0 and tableau[i, 0] / tableau[i, j + 1] >= 0:
                        if tableau[i, 0] / tableau[i, j + 1]  < tableau[theta, 0] / tableau[theta, j + 1]:
                            theta = i
#                         elif tableau[i, 0] / tableau[i, j + 1]  == tableau[theta, 0] / tableau[theta, j + 1]:
#                             if i in basic_columns:
#                                 continue
#                             else:
#                                 theta = i
                print(basic_columns)
                basic_columns[theta - 1] = j
#                 basic_columns.insert(theta - 1, j)
                pivot_row = theta          # index of direction which will exit the basis matrix
                pivot_col = j + 1          # direction which will enter the basis
                print(pivot_row, pivot_col)
                tableau[pivot_row, :] = tableau[pivot_row, :] / tableau[pivot_row, pivot_col]
                print("before",tableau[0, 0])
                for i in range(tableau.shape[0]):
                    if i == pivot_row:
                        continue        
                    tableau[i, :] = tableau[i, :] - (tableau[i, pivot_col] / tableau[pivot_row, pivot_col]) * tableau[pivot_row, :]
                print("after",tableau[0, 0])
                print(basic_columns, tableau[pivot_row, pivot_col])
                print(tableau)
                iters+=1
                if iters == self.max_iter:
                    print("\n\nCycling encountered! Method could not converge in max_iter = %d iterations. Restarting with new BFS!\n\n" %(self.max_iter))
                    self.basic_columns.pop(self.basic_columns.index(basic_columns))
                    flag = 1                             # not the right initial BFS!
                    break
                    
            if flag == 0:
                self.solution = LPSolution(self.num_vars, {basic_columns[i]: tableau[1:, 0][i] for i in range(len(basic_columns))}, tableau[0, 0])
        
                return self.solution    


In [419]:
simplex = Simplex(14, 20, np.array([-1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [420]:
simplex.fetch_constraints("flow1.dat")

In [421]:
sol = simplex.run_simplex()

[[19.  0.  0.  0.  0.  0.  1.  0. -1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [12.  1.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 7.  0.  1.  1. -1.  0.  1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 7.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [12.  0.  0.  0.  0.  0.  1.  0. -1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 4.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 6.  0.  0. -1.  1.  0. -1.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0. -1.]
 [10.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 9.  0.  0.  0.  0.  0.  1.  0.  0.  0.  

In [422]:
print(str(simplex.solution))

	Solution to the LP is as follows:

		c'x*	=	26.0

		x_1	=	13.0
		x_2	=	13.0
		x_3	=	1.0
		x_4	=	0.0
		x_5	=	12.0
		x_6	=	0.0
		x_7	=	14.0
		x_8	=	7.0
		x_9	=	19.0
		x_10	=	7.0
		x_11	=	3.0
		x_12	=	0.0
		x_13	=	9.0
		x_14	=	4.0
		x_15	=	0.0
		x_16	=	9.0
		x_17	=	0.0
		x_18	=	0.0
		x_19	=	1.0
		x_20	=	0.0



In [391]:
simplex2 = Simplex(5, 9, np.array([0.0,0,0,-2,0,0,0,0,-1]))
simplex2.fetch_constraints("constraints3.dat")
sol2 = simplex2.run_simplex()
print(str(sol2))

[[ 1.9394e+02  0.0000e+00 -2.4242e-01  4.4409e-16  0.0000e+00  0.0000e+00  4.8485e-01 -2.2204e-16  3.6364e-01  9.0909e-02]
 [ 6.6667e+01  1.0000e+00  6.6667e-01  2.0817e-17  0.0000e+00  0.0000e+00  6.6667e-01 -5.5511e-17  4.1633e-17 -2.7756e-17]
 [ 2.4242e+01  0.0000e+00 -3.0303e-02  1.0000e+00  0.0000e+00  0.0000e+00  6.0606e-02 -2.7756e-17  5.4545e-01 -3.6364e-01]
 [ 9.6970e+01  0.0000e+00 -1.2121e-01  2.2204e-16  1.0000e+00  0.0000e+00  2.4242e-01 -1.1102e-16  1.8182e-01  5.4545e-01]
 [ 3.3333e+01  5.5511e-17 -6.6667e-01  0.0000e+00  0.0000e+00  1.0000e+00 -6.6667e-01  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 1.0909e+02  0.0000e+00  1.8636e+00  0.0000e+00  0.0000e+00  0.0000e+00  2.7273e-01  1.0000e+00 -5.4545e-01  3.6364e-01]]
[0, 2, 3, 4, 6]
5 2
before 193.9393939393939
after 208.130081300813
[0, 2, 3, 4, 1] 1.0
[[ 2.0813e+02  0.0000e+00  0.0000e+00  4.4409e-16  0.0000e+00  0.0000e+00  5.2033e-01  1.3008e-01  2.9268e-01  1.3821e-01]
 [ 2.7642e+01  1.0000e+00  0.0000e+00  2.0817e-17 

In [429]:
simplex3 = Simplex(4, 4, np.array([1, 1, 1, 0.0]))
simplex3.fetch_constraints("constraints4.dat")
sol3 = simplex3.run_simplex2()
print(str(sol3))

RuntimeError: No initial non-degenrate BFS detected! Terminating algorithm...

In [274]:
B = simplex.A[:,[0, 1, 4, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18]]

In [275]:
X = np.dot(np.linalg.inv(B), A)

In [277]:
c_B = np.array([-1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])[[0, 1, 4, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18]]
Y = np.dot(c_B, X)
c = np.array([-1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
c - Y

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.])

In [215]:
C = np.loadtxt("flow1.dat")

In [216]:
A = C[:14]

In [217]:
b = A[:, -1]
A = A[:, :-1]

In [218]:
A, b, A.shape

(array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 

In [219]:
basic_columns = None
for i in itertools.combinations(range(A.shape[1]), A.shape[0]):
    B_poss = A[:, list(i)]                       # Possible B
    pivots = np.linalg.matrix_rank(B_poss)
    if pivots == B_poss.shape[1]:
        if basic_columns is None:
            basic_columns = list()
        x_b = np.linalg.solve(B_poss, b)
        if (x_b > 0).all():
            basic_columns.append(list(i))
        else:
            continue

if basic_columns is None:
    raise RuntimeError("No initial non-degenrate BFS detected! Terminating algorithm")
basic_columns = random.choice(basic_columns)

In [220]:
basic_columns = list(basic_columns)
basic_columns

[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 16, 17, 18, 19]

In [221]:
B = A[:, basic_columns]

In [222]:
c = np.loadtxt("flow1_objective.dat")
c_b = c[basic_columns]
zeroth_element = -1 * np.dot(c_b, np.linalg.solve(B, b))
zeroth_col = np.linalg.solve(B, b)
zeroth_row = c - np.dot(c_b, np.matmul(np.linalg.inv(B), A))
rest = np.dot(np.linalg.inv(B), A)

In [223]:
rest.shape, zeroth_element.shape, zeroth_col.shape, zeroth_row.shape

((14, 20), (), (14,), (20,))

In [224]:
c, c_b

(array([-1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]))

In [225]:
zeroth_row, zeroth_element

(array([ 0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.]),
 8.0)

In [226]:
tableau = np.block([
    [zeroth_element, zeroth_row],
    [zeroth_col.reshape(1, 14).T, rest]
])

In [227]:
tableau

array([[ 8.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 8.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [12.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 9.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 5.,  0., -1., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 5.,  0., -1., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 8.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1., -1., 

In [228]:
basic_columns

[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 16, 17, 18, 19]

In [229]:
while (tableau[0, 1:] < 0).any():
    print(basic_columns)
    j = np.where(tableau[0, 1:] < 0)[0][0]     # incoming basis direction
    theta = 1
    for i in range(1, tableau.shape[0]): 
        if tableau[i, j + 1] > 0 and tableau[i, 0] / tableau[i, j + 1] >= 0:
            if tableau[i, 0] / tableau[i, j + 1]  < tableau[theta, 0] / tableau[theta, j + 1]:
                theta = i
    basic_columns.pop(theta - 1)
    basic_columns.insert(theta - 1, j)
    pivot_row = theta          # index of direction which will exit the basis matrix
    pivot_col = j + 1          # direction which will enter the basis
    print(pivot_row, pivot_col)
    tableau[pivot_row, :] = tableau[pivot_row, :] / tableau[pivot_row, pivot_col]
    print("before",tableau[0, 0])
    for i in range(tableau.shape[0]):
        if i == pivot_row:
            continue        
        tableau[i, :] = tableau[i, :] - (tableau[i, pivot_col] / tableau[pivot_row, pivot_col]) * tableau[pivot_row, :]
    print("after",tableau[0, 0])
    print(basic_columns, tableau[pivot_row, pivot_col])
    print(tableau)

[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 16, 17, 18, 19]
14 2
before 8.0
after 10.0
[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 16, 17, 18, 1] 1.0
[[10.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  1.]
 [ 8.  1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 9.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 7.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 3.  0.  0.  0.  0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 8.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -1.  0.  0.  0.  0.  0.]
 [11.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0. -1.  0.  1.  0.  0.  0. -1.]


/Users/justachetan/work/acad/sem6/lo/codes/.virtualenvs/lo/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


 21.0
after 13.0
[2, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1] 1.0
[[13. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-8. -1. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -1. -0. -0. -0. -0. -0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [13.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  1.  0.  0.  0.]
 [14.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 6. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0. -1.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [16.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-4. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  1. -1.  0.  0.  0.]
 [18.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.

 [13.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]
[2, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1]
1 1
before 13.0
after 21.0
[0, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1] 1.0
[[21.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0. -1.  1.  0.  0.  0.  0.  0.]
 [ 8.  1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 5.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  1.  0.  0.  1.  0.  0.  0.]
 [14.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [14.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0. -1.  1.  0.  0.  0.  0. -1.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 8.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -1.  0.  0.  0.  0.  0.]
 [

[0, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1]
1 3
before 21.0
after 13.0
[2, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1] 1.0
[[13. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-8. -1. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -1. -0. -0. -0. -0. -0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [13.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  1.  0.  0.  0.]
 [14.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 6. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0. -1.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [16.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-4. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  1. -1.  0.  0.  0.]
 [1

...

[2, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1] 1.0
[[13. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-8. -1. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -1. -0. -0. -0. -0. -0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [13.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  1.  0.  0.  0.]
 [14.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 6. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0. -1.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [16.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-4. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  1. -1.  0.  0.  0.]
 [18.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  1.  0.  0.  0. 

after 13.0
[2, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1] 1.0
[[13. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-8. -1. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -1. -0. -0. -0. -0. -0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [13.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  1.  0.  0.  0.]
 [14.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 6. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0. -1.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [16.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-4. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  1. -1.  0.  0.  0.]
 [18.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  1.  

after 13.0
[2, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1] 1.0
[[13. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-8. -1. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -1. -0. -0. -0. -0. -0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [13.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  1.  0.  0.  0.]
 [14.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 6. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0. -1.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [16.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-4. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  1. -1.  0.  0.  0.]
 [18.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  1.  

 [13.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]
[2, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1]
1 1
before 13.0
after 21.0
[0, 3, 4, 5, 6, 8, 9, 10, 15, 12, 7, 17, 18, 1] 1.0
[[21.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0. -1.  1.  0.  0.  0.  0.  0.]
 [ 8.  1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [12.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 5.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  1.  0.  0.  1.  0.  0.  0.]
 [14.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [14.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0. -1.  1.  0.  0.  0.  0. -1.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 8.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -1.  0.  0.  0.  0.  0.]
 [

KeyboardInterrupt: 

In [ ]:
pivot_row, pivot_col

In [ ]:
simplex.c

In [ ]:
a = np.eye(10, dtype=np.int64)
b = np.eye(10, dtype=np.int64)
c = np.hstack([a, b])
d = np.array([16, 13, 10, 4, 12, 9, 14, 7, 20, 7])
e = np.hstack([c, d.reshape(d.shape[0], 1)])
for i in e: 
    for j in i: 
        print(j, end=" ")
    print("")

In [ ]:
f = np.eye(20, dtype=np.int64)
g = np.zeros(20, dtype=np.int64).reshape((f.shape[0], 1))
h = np.hstack([f, g])
for i in h: 
    for j in i: 
        print(j, end=" ")
    print("")

In [ ]:
a = "4"
a+="5"

In [72]:
np.zeros(20, dtype=np.int64)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [313]:
a = [(1, 2), (3, 4), (-1, 4)]

In [314]:
a

[(1, 2), (3, 4), (-1, 4)]

In [317]:
a.sort(key=operator.itemgetter(0))

In [318]:
a

[(-1, 4), (1, 2), (3, 4)]

In [319]:
{i: i for i in range(10)}

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

In [377]:
for i in np.hstack([np.eye(9), np.zeros(9).reshape((9, 1))]):
    for j in i:
        print(str(j), end=" ")
    print()

1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 


In [370]:
np.zeros(9).reshape((9, 1))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

99.99999999999999

In [398]:
tab = simplex.tableau.copy()
num_vars = 20

In [399]:
s = ""
for i in range(num_vars):
    s+="x_" +str(i+1)+"\t"
s+="\n"

In [402]:
print(s)

x_0	x_1	x_2	x_3	x_4	x_5	x_6	x_7	x_8	x_9	x_10	x_11	x_12	x_13	x_14	x_15	x_16	x_17	x_18	x_19	



In [406]:
a1 = np.hstack([np.eye(29), np.eye(29)])
a2 = np.array([11, 15, 10, 18, 4, 3, 5, 6, 3, 11, 4, 17, 6, 3, 13, 12, 4, 21, 4, 9, 4, 3, 4, 5, 4, 7, 9, 2, 15])
a3 = np.hstack([a1, a2.reshape(a2.shape[0], 1)])

In [409]:
for i in a3:
    for j in i:
        print(j, end=",")
    print()

1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,
0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,
0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,
0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,
0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [411]:
for i in np.zeros((58, 58)):
    for j in i:
        print(j, end=" ")
    print()

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 


In [414]:
print(np.zeros(58).tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
